# Twitter comment sentiment analysis Project
This projetc aims to understand +ve or -ve intentions behind a twitter comment.
## 1] Convert Data to make it ready for ML model
convert data in terms which model can understand (which is mostly numbers or csv??) next split it into train and test
## 2] Use suitable models and cross validate to improve prediction
This prediction is not only a result of a simple input output relation this output is a binary number and input can be infinitely anything.
## 3] Plot results

In [ ]:
! pip import kaggle


ERROR: unknown command "import"


In [ ]:
!mkdir -p ~/.kaggle
!cp /kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat '/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
# use API token to get dataset
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 88% 71.0M/80.9M [00:00<00:00, 124MB/s] 
100% 80.9M/80.9M [00:00<00:00, 117MB/s]


In [ ]:
# Extract data from zipped file
from zipfile import ZipFile

with ZipFile("/content/sentiment140.zip","r") as zip:
  zip.extractall()
  print("Extract check")

Extract check


# Import libs

In [ ]:
import numpy as np
import pandas as pd
import re # Google this, know more about this
# re is regular expression
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


In [ ]:
# importing all stopwords
import nltk
nltk.download("stopwords")
print(stopwords.words("english"))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# nltk.corpus
this is a natural language processing library corpus is the module from wherein we import stopwords
### stopwords are list of words which do not add any influential meaning to our sentence
# nltk.stem.porter
stem.porter and PorterStemmer is a way to reduce the words to its root words to simplify processing (**Google this as well**)
# TfidfVectorizer
this function allows us to convert the text into numbers which are easier for the computer to understand

# Import Models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

# Data Processing

In [ ]:
#Load data in pd while giving column names
column_names=["target","id","date","user","flag","text"]
df=pd.read_csv("/content/training.1600000.processed.noemoticon.csv",names=column_names, encoding="ISO-8859-1")
print(df.head())


   target          id                          date      user  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              flag                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....  


## As visible the column names are missing hence we will give column name ourselves

In [ ]:
filtered_df=df.dropna(axis=0) # To remove any empty rows
filtered_df.isnull().sum() #isnull() counts and sum() adds up This step is to check for empty cells if any
filtered_df["target"].value_counts() # value_counts() gives sum count of each different value

target
0    800000
4    800000
Name: count, dtype: int64

## Our data is equal in quantity for target
## If it were not we would have to face problem of upsampling and downsampling

In [ ]:
#Convert 4 to 1 as only 0 and 4 are there and its better to have 1 Id say

# Try keeping 4 as 4 only and see difference in results

In [ ]:
filtered_df.replace({"target":{4:1}} , inplace=True)

# Stemming
Process to simplify the word to a root word
Ex: actor, actress, acting all are reduced to act
###This step is done to simplify the process and reduce complexity
##Will this step also introduce some error??


In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content= re.sub("[^a-zA-Z]"," ",content) #The upper arrow removes whatever is not a-z or A-Z
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words("english") ]
    stemmed_content=' '.join(stemmed_content)

    return stemmed_content

In [ ]:
filtered_df["stemmed_content"]=filtered_df["text"].apply(stemming)

KeyboardInterrupt: 

In [ ]:
filtered_df.head()

In [ ]:
X=filtered_df["stemmed_content"].values
Y=filtered_df["target"].values
print(X,Y)


## x=df["coulmn_name"] This keeps the format of X in df only
## x=df["coulmn_name"].values This converts all the values in column into a list and store it in X

In [ ]:
train_X,val_X,train_y,val_y=train_test_split(X,Y,test_size=0.25,random_state=1,stratify=Y)

## stratify=Y makes sure the split has been so that 0 and 1 remain equal in train and test

# Convert text to numerical values

In [ ]:
vectorizer=TfidfVectorizer()

train_X=vectorizer.fit_transform(train_X)
val_X=vectorizer.transform(val_X)

## the val_X is not undergoing fitting because fitting already looks at textual data in train_X and then assigns importance value accordingly, later the val_X is similarly transform as it transforms train_X

#ML Logistic regression part

In [ ]:
model=LogisticRegression(max_iter=7000)

In [ ]:
model.fit(train_X,train_y)

In [ ]:
val_predictions=model.predict(val_X)
accuracy=accuracy_score(val_y,val_predictions)
print(accuracy)

### the train and test accuracy are close means model performs well
## If the test data accuracy<<train data accuracy => model is overfitted

In [ ]:
import pickle

In [ ]:
filename="model_twitter_logistic.sav"
pickle.dump(model,open(filename,"wb")) #wb is writing in binary. Binary so that it reduces space